# Setup


In [ ]:
%pip install "ase@git+https://gitlab.com/ase/ase.git" "matcalc[matgl]>=0.4.5"

# Objective


Here, we will demonstrate how to model surfaces. We will study the adsorption of N2 on Fe. For this analysis, we will use the TensorNet-MatPES-r2SCAN machine learning potential we used in a prior exercise. Again, the choice of calculator can be easily swapped and is not the main focal point of this exercise.

The question we want to answer is: how strongly does N2 adsorb on the surface of iron, and what is the most stable adsorption geometry?


In [ ]:
potential_name = "TensorNet-MatPES-r2SCAN-v2025.1-PES"

# Optimizing the Bulk Structure


Our first order of business is to construct a surface of Fe. We start by taking the bulk structure of Fe and relaxing it. The bulk structure of Fe could be obtained from an experimental crystal structure, from the Materials Project, or any other place. We will take the one provided by ASE for convenience.


In [ ]:
from ase.build import bulk

crystal = bulk("Fe")

Now we attach the calculator and relax the bulk Fe structure. We make sure to relax both the positions and unit cell here.


In [ ]:
from ase.filters import FrechetCellFilter
from ase.optimize import BFGS
from matcalc import load_fp

crystal.calc = load_fp(potential_name)  # Assign a calculator
crystal_wrapped = FrechetCellFilter(crystal)  # Tell ASE to optimize cell too
opt = BFGS(crystal_wrapped, trajectory="bulk_relax.traj")  # Set up optimizer
opt.run(fmax=0.01)  # Run optimization until forces < 0.01 eV/Å

# Carving and Optimizing the Surfaces


Now that we have a relaxed bulk structure, we can carve a surface. There are many tools to carve surfaces in Pymatgen, but I have already coded some convenient utilities in my code quacc. We will use those for simplicity. As always, we should look at the arguments and docstrings to understand what the function does.


In [ ]:
from quacc.atoms.slabs import make_slabs_from_bulk

slabs = make_slabs_from_bulk(crystal)

We have generated a bunch of plausible surface slabs. Let's view some of them. Notice how some of the bottom layers are held fixed. If you wanted to do this yourself, you could do so with the `ase.constraints.FixAtoms` class (for details, refer to the ASE documentation on [constraints](https://ase-lib.org/ase/constraints.html)).


In [ ]:
from ase.visualize import view

view(slabs)

Now that we have a bunch of surface slabs, which do we pick?

We generally must find the one with the lowest energy. So, now we must relax each surface _at a fixed cell volume_ (i.e. the one from the bulk structure optimization) and find the surface with the lowest energy.

Technically, we should identify the surface with the lowest surface energy rather than total energy. The surface energy is the work required to to carve a surface with a given surface area from a bulk sample and has units of eV/A^2. The surface energy is given by

$$\gamma = \frac{E_{\mathrm{slab}} - \frac{N_{\mathrm{slab}}}{N_{\mathrm{bulk}}} E_{\mathrm{bulk}}}{2 A}$$
where $E$ are the total energies of the surface slab and bulk, $N$ is the number of atoms, and $A$ is he surace area of the slab.

However, for simplicity we will simply compare the total energies of the slabs for simplicity and because it does not change our demonstration. Remember that we cannot compare energies if the compositions are different, so we will sstill have to normalize by the number of atoms.


In [ ]:
for slab in slabs:
    slab.calc = load_fp(potential_name)
    opt = BFGS(slab)
    opt.run(fmax=0.01)

In [ ]:
for i, slab in enumerate(slabs):
    e = slab.get_potential_energy() / len(slab)
    print(f"Slab {i} energy: {e} eV/atom")

We will continue with the lowest energy surface here.


In [ ]:
import numpy as np

slab_energies = [slab.get_potential_energy() for slab in slabs]
min_energy_index = np.argmin(slab_energies)
slab = slabs[0]

# Adding Adsorbates


Now we need to add an adsorbate to our slab. Let's start by defining the N2 molecule.


In [ ]:
from ase.build import molecule
from ase.optimize import BFGS

adsorbate = molecule("N2")
adsorbate.calc = load_fp(potential_name)
opt = BFGS(adsorbate)
opt.run(fmax=0.01)

Now we need to add our adsorbate to our surface. There are many possible surface sites. The only way to know where the adsorbate should go is to add the adsorbate to all plausible surface sites, do a structure relaxation, and identify the lowest energy configuration(s). There are tools in Pymatgen to do this, but again we will use a utility function I have developed in quacc.


In [ ]:
from quacc.atoms.slabs import make_adsorbate_structures

slab_adsorbates = make_adsorbate_structures(slab, adsorbate)

We now have many slab-adsorbate systems with the adsorbate at various sites.


In [ ]:
view(slab_adsorbates)

Time to relax each one to find the lowest energy configuration for the adsorbate. We do not need to normalize on a per-atom basis here because all the systems have the same number of atoms since we are using the same surface for each calculation.


In [ ]:
from ase.optimize import BFGS

for slab_adsorbate in slab_adsorbates:
    slab_adsorbate.calc = load_fp(potential_name)
    opt = BFGS(slab_adsorbate)
    opt.run(fmax=0.01)

In [ ]:
for i, slab_adsorbate in enumerate(slab_adsorbates):
    print(f"System {i} energy: {slab_adsorbate.get_potential_energy()} eV")

In [ ]:
view(slab_adsorbates)

We will continue with the lowest energy configuration.


In [ ]:
min_idx = np.argmin(
    [slab_adsorbate.get_potential_energy() for slab_adsorbate in slab_adsorbates]
)
final_system = slab_adsorbates[min_idx]

In [ ]:
view(final_system)

# Confirming We Have a Minimum Energy Configuration


We can be thorough and confirm that the structure we have is indeed a local minimum by carrying out a vibrational mode analysis.


In [ ]:
from ase.vibrations import Vibrations

vib = Vibrations(final_system)
vib.clean()  # make sure we start fresh
vib.run()  # run vibrational analysis
vib.summary()  # report results

No imaginary modes. Excellent!


# Calculating the Adsorption Energy


Now, we can calculate the adsorption energy.


In [ ]:
delta_E = (
    final_system.get_potential_energy()
    - slab.get_potential_energy()
    - adsorbate.get_potential_energy()
)
print(f"Adsorption energy: {delta_E} eV")

Experimentally, it is known that Fe will dissociate N2 under typical reaction conditions. At low temperatures (~140 K), molecular adsorption (i.e. without dissociation) occurs. For reference, on the Fe(111) surface, the N2 adsorption energy is estimated to be between -0.22 eV to -0.43 eV. Source: https://doi.org/10.1016/0021-9517(77)90237-8.
